# MODELO SOLUCIÓN: LIGHTGBM

Importación de Librerías y Definición de Mapeos Se instalan las dependencias necesarias (LightGBM) y se definen diccionarios de configuración (CFG_MAPS) para transformar variables categóricas ordinales (como educación y estratos) en valores numéricos interpretables por el modelo.

In [ ]:
!pip install lightgbm -q --upgrade

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc
import warnings

# Silenciar advertencias innecesarias
warnings.filterwarnings('ignore')

# DICCIONARIOS DE CONFIGURACIÓN
CFG_MAPS = {
    'binario': {'Si': 1, 'No': 0, 'S': 1, 'N': 0},
    'educacion': {
        'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2,
        'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4,
        'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6,
        'Postgrado': 7
    },
    'estrato': lambda x: str(x).replace('Estrato ', '').replace('Sin Estrato', '0').replace('nan', '0'),
    'matricula': {
        'No pagó matrícula': 0, 'Menos de 500 mil': 1,
        'Entre 500 mil y menos de 1 millón': 2, 'Entre 1 millón y menos de 2.5 millones': 3,
        'Entre 2.5 millones y menos de 4 millones': 4, 'Entre 4 millones y menos de 5.5 millones': 5,
        'Entre 5.5 millones y menos de 7 millones': 6, 'Más de 7 millones': 7
    },
    'trabajo': {
        '0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2,
        'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4
    }
}

Ingesta de Datos desde API de Kaggle Configuración del entorno para descargar el dataset directamente desde la competición utilizando la API de Kaggle, asegurando acceso rápido y reproducible a los archivos train.csv y test.csv.

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
!chmod 600 kaggle.json 2>/dev/null
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia --force
!unzip -o udea*.zip > /dev/null
print("Archivos disponibles:")
!ls *.csv

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 522MB/s]
Archivos disponibles:
submission_example.csv	test.csv  train.csv


Preprocesamiento e Ingeniería de Características Función central que limpia los datos, aplica los mapeos definidos, gestiona valores nulos y crea nuevas variables sintéticas (como Capital Educativo). Finalmente, prepara las matrices X (features) y y (target) para el entrenamiento.

In [ ]:
def feature_engineering(df_raw):
    """
    Transforma el dataframe aplicando mappings y creando nuevas variables.
    """
    df = df_raw.copy()

    # 1. Limpieza de Estrato (Aplicando la lambda definida arriba)
    if 'F_ESTRATOVIVIENDA' in df.columns:
        df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].apply(CFG_MAPS['estrato']).astype(float)

    # 2. Mapeos Ordinales (Educación, Matrícula, Trabajo)
    col_mappings = {
        'F_EDUCACIONPADRE': CFG_MAPS['educacion'],
        'F_EDUCACIONMADRE': CFG_MAPS['educacion'],
        'E_VALORMATRICULAUNIVERSIDAD': CFG_MAPS['matricula'],
        'E_HORASSEMANATRABAJA': CFG_MAPS['trabajo']
    }

    for col, mapping in col_mappings.items():
        if col in df.columns:
            # Usamos map, pero llenamos NaN con -1 para que el modelo sepa que falta
            df[col] = df[col].map(mapping).fillna(-1)

    # 3. Variables Binarias (Detección automática de columnas por nombre)
    cols_binarias = [c for c in df.columns if 'TIENE' in c or 'PRIVADO' in c or 'PAGO' in c]
    for col in cols_binarias:
        df[col] = df[col].map(CFG_MAPS['binario']).fillna(-1)

    # 4. Creación de Características (Feature Creation)
    # Suma de capital educativo (Padre + Madre)
    if 'F_EDUCACIONPADRE' in df.columns and 'F_EDUCACIONMADRE' in df.columns:
        df['S_CAPITAL_EDUCATIVO'] = df['F_EDUCACIONPADRE'] + df['F_EDUCACIONMADRE']

    # Optimización de Tipos
    # Convertir columnas tipo 'object' a 'category' es CRÍTICO para LightGBM
    cat_cols = df.select_dtypes(include=['object']).columns
    for col in cat_cols:
        if col not in ['ID', 'RENDIMIENTO_GLOBAL']:
            df[col] = df[col].astype('category')

    return df

# Carga y proceso
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Eliminación de columnas constantes (varianza 0)
const_cols = [c for c in train.columns if train[c].nunique(dropna=False) <= 1]
train.drop(columns=const_cols, inplace=True)
test.drop(columns=const_cols, errors='ignore', inplace=True)

# Transformación
X_full = feature_engineering(train.drop(columns=['RENDIMIENTO_GLOBAL', 'ID']))
y_full = train['RENDIMIENTO_GLOBAL']
X_test = feature_engineering(test.drop(columns=['ID']))

# Encoding del Target
le = LabelEncoder()
y_encoded = le.fit_transform(y_full)

print(f"Features finales: {X_full.shape[1]}")

Features finales: 20


Entrenamiento con Validación Cruzada Estratificada (Stratified K-Fold) Configuración de hiperparámetros de LightGBM y entrenamiento iterativo utilizando 5 pliegues (Folds). Esto genera predicciones más robustas y evita el sobreajuste (overfitting).

In [ ]:
# Parámetros optimizados para estabilidad
LGBM_PARAMS = {
    'objective': 'multiclass',
    'num_class': len(le.classes_),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.035,   # Ligeramente más bajo para mayor precisión
    'num_leaves': 45,
    'max_depth': 10,
    'feature_fraction': 0.8,  # Selecciona 80% de features por árbol (evita overfit)
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'n_jobs': -1,
    'verbose': -1,
    'seed': 42
}

# Estrategia: Stratified K-Fold
FOLDs = 5
skf = StratifiedKFold(n_splits=FOLDs, shuffle=True, random_state=42)

# Matrices para guardar resultados
oof_preds = np.zeros((len(X_full), len(le.classes_)))
test_preds = np.zeros((len(X_test), len(le.classes_)))

print(f"Iniciando entrenamiento con {FOLDs} Folds...")

for fold, (train_idx, val_idx) in enumerate(skf.split(X_full, y_encoded)):
    # Split de datos
    X_train, y_train = X_full.iloc[train_idx], y_encoded[train_idx]
    X_val, y_val = X_full.iloc[val_idx], y_encoded[val_idx]

    # Crear datasets de LightGBM
    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

    # Entrenar
    model = lgb.train(
        LGBM_PARAMS,
        dtrain,
        num_boost_round=2000, # Damos mucho espacio
        valid_sets=[dtrain, dval],
        callbacks=[
            lgb.early_stopping(stopping_rounds=50, verbose=False),
            lgb.log_evaluation(period=0) # Silenciar log masivo
        ]
    )

    # Predecir validación y test
    oof_preds[val_idx] = model.predict(X_val)
    test_preds += model.predict(X_test) / FOLDs # Promediamos las predicciones

    # Score del fold actual
    acc_fold = accuracy_score(y_val, np.argmax(oof_preds[val_idx], axis=1))
    print(f"Fold {fold+1}: Accuracy = {acc_fold:.4f}")

# Evaluación Global
acc_global = accuracy_score(y_encoded, np.argmax(oof_preds, axis=1))
print(f"\n--- Accuracy Promedio Global: {acc_global:.5f} ---")

Iniciando entrenamiento con 5 Folds...
Fold 1: Accuracy = 0.4416
Fold 2: Accuracy = 0.4390
Fold 3: Accuracy = 0.4390
Fold 4: Accuracy = 0.4414
Fold 5: Accuracy = 0.4405

--- Accuracy Promedio Global: 0.44030 ---


Generación del Archivo de Submission Decodificación de las predicciones numéricas a sus etiquetas originales y creación del archivo .csv final para subir a la plataforma.

In [ ]:
# Convertir probabilidades a etiquetas finales
predicciones_finales_indices = np.argmax(test_preds, axis=1)
etiquetas_finales = le.inverse_transform(predicciones_finales_indices)

# Guardar
nombre_sub = 'submission_kfold_ensemble.csv'
submission = pd.DataFrame({
    'ID': test['ID'],
    'RENDIMIENTO_GLOBAL': etiquetas_finales
})

submission.to_csv(nombre_sub, index=False)
print(f"Archivo generado exitosamente: {nombre_sub}")

Archivo generado exitosamente: submission_kfold_ensemble.csv
